In [1]:
import pandas as pd
import numpy as np

In [2]:
insample = pd.read_csv('./result/pred_in_sample_linear.csv')

In [17]:
outsample = pd.read_csv('./result/All_results.csv')

In [18]:
outsample

,Unnamed: 0,id,name,date,return,pred_lasso,pred_ridge,pred_elasticnet,APT,lightGBM,XGBoost,NN,CNN
0,1459,338,潍柴动力,2022-01-04,0.009610,0.000867,0.000192,0.000872,-0.000276,-0.004144,0.004798,0.117966,0.105475
1,1460,338,潍柴动力,2022-01-05,0.003359,0.000862,-0.000114,0.000867,-0.000626,-0.003142,0.007397,0.117791,0.105533
2,1461,338,潍柴动力,2022-01-06,-0.008929,0.000861,-0.000046,0.000865,-0.000505,-0.003652,0.007397,0.117796,0.105531
3,1462,338,潍柴动力,2022-01-07,0.016892,0.000865,-0.000017,0.000870,-0.000584,0.001137,0.007029,0.117759,0.105217
4,1463,338,潍柴动力,2022-01-10,-0.026578,0.000856,-0.000250,0.000861,-0.000807,-0.002752,0.008083,0.117647,0.104820
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2156,16973,601689,拓普集团,2022-12-23,0.078448,0.001246,0.001590,0.001245,0.001436,0.001412,0.006535,0.118037,0.121555
2157,16974,601689,拓普集团,2022-12-26,-0.001290,0.001235,0.001535,0.001235,0.001222,0.003983,0.000977,0.117904,0.121038
2158,16975,601689,拓普集团,2022-12-27,-0.059916,0.001236,0.001775,0.001235,0.001396,0.002365,0.005349,0.117811,0.120082
2159,16976,601689,拓普集团,2022-12-28,0.006356,0.001244,0.001596,0.001244,0.001077,0.013397,0.002658,0.118212,0.119489


In [7]:
insample

,id,name,date,return,Baseline,Lasso,Ridge,ENet
0,338,潍柴动力,2016-01-04,-0.012236,0.003308,0.001857,0.003293,0.001857
1,338,潍柴动力,2016-01-05,0.036036,0.003337,0.001849,0.003322,0.001848
2,338,潍柴动力,2016-01-06,-0.088043,0.003229,0.001833,0.003216,0.001832
3,338,潍柴动力,2016-01-07,0.030989,0.002882,0.001782,0.002863,0.001781
4,338,潍柴动力,2016-01-08,-0.036994,0.002803,0.001776,0.002786,0.001776
...,...,...,...,...,...,...,...,...
14554,601689,拓普集团,2021-12-27,-0.006483,0.000526,0.000872,0.000532,0.000872
14555,601689,拓普集团,2021-12-28,-0.013609,0.000960,0.000880,0.000964,0.000880
14556,601689,拓普集团,2021-12-29,0.020601,0.001380,0.000878,0.001382,0.000878
14557,601689,拓普集团,2021-12-30,-0.018519,0.001495,0.000924,0.001496,0.000924


In [4]:
def cal_ICIR(data: pd.DataFrame, feild: str) -> tuple[float, float]:
    """
    data is a dataframe with columns: date, return, factor feild
    feild is the factor name
    return IC and IR
    """
    data = data.loc[:, ['date', 'return', feild]]
    data.dropna(inplace=True)
    IC_dataframe = data.groupby('date').apply(lambda x: x.corr(method='spearman')[feild]['return'])
    return IC_dataframe.mean(), IC_dataframe.mean()/IC_dataframe.std()

def test_factor(ICIR: tuple[float, float]) -> str:
    """
    ICIR is a tuple of IC and IR
    return the test result
    """
    if abs(ICIR[0]) > 0.01 and abs(ICIR[1]) > 0.03:
        return 'pass'
    else:
        return 'fail'

In [36]:
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score

In [39]:
r2_score(outsample.loc[:, "return"], outsample.loc[:, "pred_lasso"])

-0.0029964995516744697